In [8]:
import cv2
from skimage import io
import numpy as np
from PIL import Image 
import os

In [63]:
#this function takes the whole image and returns a list of images, each image represent a handwriting text line 
def lines_segments(imagefile):
    image = cv2.imread(imagefile)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    edged = cv2.Canny(gray, 30, 200) 
    lst=edged.sum(axis=1)
    x1=lst[650]
    for i in range(651,850):
        if lst[i]>x1:
            x1=i

    x2=lst[2000]
    for i in range(2000,2800):
        if lst[i]>x2:
            x2=i
    #cv2.imwrite('out.png', image[x1:x2,:])
    image = image[x1:x2,:]
    #grayscale
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    #binary
    ret,thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY_INV)
    #dilation
    kernel = np.ones((5,100), np.uint8)
    img_dilation = cv2.dilate(thresh, kernel, iterations=1)
    #find contours
    ctrs, hier = cv2.findContours(img_dilation.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #sort contours
    sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
    images = []
    for i, ctr in enumerate(sorted_ctrs):
        # Get bounding box
        x, y, w, h = cv2.boundingRect(ctr) 
        # Getting ROI
        roi = image[y:y+h, x:x+w]
        if(len(roi)>20): #20 is thershold you can change this, this is for small lines and points, remove them from return images 
            images.append(roi)
    # write
    return images

In [64]:
images = lines_segments("test.png")